In [7]:
from dataclasses import dataclass


@dataclass
class Letter:
    text: str
    contains: bool
    correct: bool
    
    @property
    def color(self):
        if self.correct or self.contains:
            return (0, 0, 0)
        else:
            return (255, 255, 255)

    @property
    def block_file(self):
        if self.correct:
            return "../assets/correct.png"
        elif self.contains:
            return "../assets/contains.png"
        else:
            return "../assets/wrong.png"


@dataclass
class Word:
    text: str
    letters: list[Letter]
    
    @property
    def correct(self) -> bool:
        for letter in self.letters:
            if not letter.correct:
                return False
        
        return True

In [8]:
from PIL import Image
from PIL import ImageFont
from PIL import ImageDraw
from io import BytesIO

class WordleManager:
    def __init__(self, correct_word, base_file="../assets/game.png",
                 font_file="../assets/font.ttf", font_size=72):
        
        self.correct_word = correct_word
        self.img = Image.open(base_file)
        self.draw = ImageDraw.Draw(self.img)
        self.font = ImageFont.truetype(font_file, font_size)
        
        # Базовые координаты (0, 0)
        self.base_x = 21
        self.base_y = 18
        
        # Сдвиги для блока с буквой
        self.block_x_offset = 93
        self.block_y_offset = 103
        
        # Сдвиги для буквы относительно блока
        self.text_x_offset = 16
        self.text_y_offset = -3
        
        # Счётчик слов на игровом поле
        self.word_count = 0
    
    def prepare_word(self, word: str) -> Word:
        word_obj = Word(word, [])
        for index, letter in enumerate(word):
            correct = False
            contains = False
            
            # Проверка на наличие буквы в слове
            if letter in self.correct_word:
                contains = True
            
                # Проверка на правильную позицию буквы в слове
                if letter == self.correct_word[index]:
                    correct = True
            
            word_obj.letters.append(Letter(letter, contains, correct))
            
        return word_obj
    
    def draw_word(self, word: Word):
        if self.word_count == 6:
            raise EOFError('Игровое поле уже заполнено.')
        
        letter_index = 0
        
        
        
        for letter in word.letters:
            # print(f"Пишем букву «{letter.text}»")
            # Открываем файл с нужным блоком для буквы
            block = Image.open(letter.block_file)
            
            # Вычисляем координаты блока
            block_x = self.base_x + letter_index * self.block_x_offset
            block_y = self.base_y + self.word_count * self.block_y_offset
            
            # Вставляем изображение блока на поле
            self.img.paste(block, (block_x, block_y))
            
            # Вычисляем координаты буквы относительно блока
            text_x = block_x + self.text_x_offset
            text_y = block_y + self.text_y_offset
            
            # Вставляем букву в блок на поле
            self.draw.text(
                xy=(text_x, text_y), 
                text=letter.text.upper(), 
                fill=letter.color, 
                font=self.font
            )
            
            letter_index += 1
        
        # Если получено правильное слово...
        if word.correct:
            # Вставляем рамку для выделения этого слова
            correct_word = Image.open('../assets/full_correct.png').convert('RGBA')
            self.img.paste(correct_word, (self.base_x-15, block_y-15), correct_word)
        
        self.word_count += 1
    
    def save(self):
        self.img.save('game-out.png')
    
    @property
    def image_bytes(self):
        output = BytesIO()
        self.img.save(output, format='PNG')
        return output


word_1 = Word(
    text="НОСОК",
    letters=[
        Letter("Н", False, False),
        Letter("О", False, False),
        Letter("С", True, False),
        Letter("О", False, False),
        Letter("К", False, False),
    ]
)

word_2 = Word(
    text="САРАЙ",
    letters=[
        Letter("С", True, False),
        Letter("А", True, False),
        Letter("Р", True, True),
        Letter("А", False, False),
        Letter("Й", False, False),
    ]
)


In [30]:
wordle = WordleManager('НОСОК')
w1 = wordle.prepare_word('САРАЙ')
w2 = wordle.prepare_word('НОЧКА')
w3 = wordle.prepare_word('КУСОК')
w4 = wordle.prepare_word('НОСОК')
wordle.draw_word(w1)
wordle.draw_word(w2)
wordle.draw_word(w3)
wordle.draw_word(w4)
wordle.save()